<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAL</center></h2>
<h3><center>3/3 : Application du NLP au df</center></h3>

<b><u>PLAN:</u></b>
1) Exploration des données
<br>
2) NER avec Doccano

    a) initialisation des données
    b) Exploration de la NER préalable de Spacy
    c) Mise en place de la stratégie pour l'annotation sur Doccano
    d) Entraînement par 5-fold cross validation

**3) Application du NLP au df**

**Enjeux de la partie 2:**

<br></br>
**L'objectif à long terme** de cet entraînement NER est de :
- Mettre en place une NER qui pourrait être appliquée au full text de chacune des revues une fois les OCR nettoyées. 
- Appliquer cette NER à la revue des Archives de médecine navale et coloniale (1892-1898)
- Alimenter une future base de données relationnelle


**Limites de mon travail:** Il me sera nécessaire de mesurer si elle fonctionne bien à la fois sur le reste de la période étudiée ainsi que sur le Bulletin de la Société de Pathologie exotique



Sources:
* https://stackoverflow.com/questions/44395656/applying-spacy-parser-to-pandas-dataframe-w-multiprocessing 


* https://towardsdatascience.com/from-dataframe-to-named-entities-4cfaa7251fc0

# 0. Initialisation

In [1]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re
import json


#Importation de mon module 
import Training_Doccano
from Training_Doccano import pandas_to_doccano

#Autres
import pickle
from collections import Counter
from os import path
from pathlib import Path
import glob


### 0.1 Import des données

In [2]:
#Import des dataframes 
dfAHMC = pd.read_csv("./Notebook_data/input/AHMC_articles-originaux_Termine2.tsv" ,sep='\t')

df_spacyAHMC = dfAHMC.loc[dfAHMC["revue_annee"]<= 1908]
df_spacyAHMC = df_spacyAHMC[["article_titre","revue_annee"]]

dfAMN=pd.read_csv("./Notebook_data/input/AMN-articles-98-08_clean-main.tsv",sep="\t")
dfAMN=dfAMN[:-2] #les deux derniers rows étaient des nan
df_spacyAMN = dfAMN[["article_titre","revue_annee"]]


print("Nombre d'articles dans chaque revue de 1898 à 1908:",
      "\nAHMC :",len(df_spacyAHMC),
      "\nAMN : ",len(df_spacyAMN),
      "\nTotal à traiter : ",len(df_spacyAHMC)+len(df_spacyAMN))


Nombre d'articles dans chaque revue de 1898 à 1908: 
AHMC : 559 
AMN :  638 
Total à traiter :  1197


### 0.2. Import du NLP entraîné

In [3]:
#NLP entraîné : 
nlp_dir = "./Notebook_data/Training_Doccano/trained_nlp/nlp1"

nlp_test1 = spacy.load(nlp_dir)


# Fonction pour appliquer le NLP aux titres du df

### Rappel des ent.label : 

*  384: 'GPE'
*  4192465394259306970: 'PATH'
*  5496441482075855828: 'SHS'
*  385: 'LOC'
*  11253504087253340061: 'CHEM-LABO'
*  383: 'ORG'

In [44]:
def NLPonDF (df):
    
    #Pour pouvoir analyser les entités dans leur ensemble
    listeEnsGPE = []
    listeEnsLOC = []
    listeEnsORG = []
    listeEnsPATH = []
    listeEnsSHS=[]
    listeEnsCHEMLABO = []
    
    df = pd.concat(
    [
        df,
        pd.DataFrame(
            [["", "", "","", "", ""]], 
            index=df.index, 
            columns=['GPE_ents', 'LOC_ents', 'ORG_ents',"PATH_ents","SHS_ents","CHEM-LABO_ents"]
        )
    ], axis=1
)
    
    
    for i in range (len(df)):
        listeGPE = []
        listeLOC = []
        listeORG = []
        listePATH = []
        listeSHS=[]
        listeCHEMLABO = []

        titre = df.iloc[i,0]
        tokenTitre = nlp_test1(titre)

        for ent in tokenTitre.ents:
            #GPE
            if ent.label_ == "GPE" :
                listeGPE.append(ent.text)
                listeEnsGPE.append(ent.text)
            GPE=','.join([str(gpe)for gpe in listeGPE])
            df.iloc[i,2] =GPE

            #LOC : 
            if ent.label == 385:
                listeLOC.append(ent.text)
                listeEnsLOC.append(ent.text)
            LOC=','.join([str(loc)for loc in listeLOC])
            df.iloc[i,3] = LOC

            #ORG : 
            if ent.label == 383:
                listeORG.append(ent.text)
                listeEnsORG.append(ent.text)
            ORG=','.join([str(org)for org in listeORG])
            df.iloc[i,4] = ORG
    
            #PATH : 
            if ent.label == 4192465394259306970:
                listePATH.append(ent.text)
            PATH = ','.join([str(path)for path in listePATH])
            df.iloc[i,5] = PATH
        
            #SHS : 
            if ent.label == 5496441482075855828:
                listeSHS.append(ent.text)
                listeEnsORG.append(ent.text)
            SHS = ','.join([str(shs)for shs in listeSHS])
            df.iloc[i,6] =SHS
    
            #CHEM-LABO : 
            if ent.label == 11253504087253340061:
                listeCHEMLABO.append(ent.text)
                listeEnsCHEMLABO.append(ent.text)
            CHEMLABO = ','.join([str(chemlabo)for chemlabo in listeCHEMLABO])
            df.iloc[i,7] =CHEMLABO
    
    return df,listeEnsGPE,listeEnsLOC,listeEnsORG,listeEnsPATH,listeEnsSHS,listeEnsCHEMLABO

# Application du NLP aux AHMC

In [46]:
df_spacyAHMCannotations,listeEnsGPE,AHMC_listeEnsLOC,AHMC_listeEnsORG,AHMC_listeEnsPATH,AHMC_listeEnsSHS,AHMC_listeEnsCHEMLABO=NLPonDF(df_spacyAHMC)

In [35]:
df_spacyAHMCannotations.to_csv("./Notebook_data/Apply_Doccano-df/dfAHMCannotations.csv",index=False)

In [48]:
pd.DataFrame(AHMC_listeEnsLOC).to_csv("./Notebook_data/Apply_Doccano-df/AHMC_listeEnsLoc.csv")

# Application NLP aux AMN

In [38]:
df_spacyAMNannotations = NLPonDF(df_spacyAMN)

In [39]:
df_spacyAMNannotations.to_csv("./Notebook_data/Apply_Doccano-df/dfAMNannotations.csv",index=False)

## Exploration sur l'ensemble des colonnes

In [9]:
#AHMC
tokensAHMC = nlp_test1(''.join(str(df_spacyAHMC.article_titre.tolist())))

#AMN
tokensAMN = nlp_test1(''.join(str(df_spacyAMN.article_titre.tolist())))

### 1.2. Extraction des entités

In [10]:
itemsAHMC = [x.text for x in tokensAHMC.ents]
itemsAMN = [x.text for x in tokensAMN.ents]

#Voir les 20 principaux:
# Counter(itemsAHMC).most_common(20)
# Counter(itemsAMN).most_common(20)